In [ ]:
# Import libraries:
import os
import time
import requests
import threading
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta

In [ ]:
load_dotenv()

In [ ]:
# Retrieve paper trading url & API keys:
alpaca_paper_endpoint=os.getenv('ALPACA_PAPER_ENDPOINT')
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

In [ ]:
# Create trading API object:
alpaca_api=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    alpaca_paper_endpoint,
    api_version='v2'
)

In [ ]:
# Retrieve account details:
account = alpaca_api.get_account()
print(account)

In [ ]:
# List Positions:
positions=alpaca_api.list_positions()
print(positions)

In [ ]:
# Put positions info into a DataFrame and export to .csv file:
positions_df=pd.DataFrame({
    'asset_id':[x.asset_id for x in positions],
    'ticker':[x.symbol for x in positions],
    'cost_basis':[x.cost_basis for x in positions],
    'asset_id':[x.asset_id for x in positions],
    'current_price':[x.asset_id for x in positions],
    'lastday_price':[x.asset_id for x in positions],
    'change_today':[x.change_today for x in positions],
    'qty':[x.qty for x in positions],
    'market_value':[x.market_value for x in positions],
    'unrealized_intraday_pl':[x.unrealized_intraday_pl for x in positions],
    'unrealized_intraday_plpc':[x.unrealized_intraday_plpc for x in positions],
    'unrealized_pl':[x.unrealized_intraday_plpc for x in positions],
    'unrealized_plpc':[x.unrealized_intraday_plpc for x in positions]
})
positions_df.to_csv('Data/stock_positions.csv')

In [ ]:
# Create tickers list from positions_df DataFrame
positions_tickers=positions_df.ticker.tolist()
print(positions_tickers)

In [ ]:
# Create the streaming connection:
# conn = tradeapi.stream2.StreamConn(alpaca_api_key, alpaca_secret_key, alpaca_paper_endpoint)
websocket_url = 'wss://data.alpaca.markets'

conn = tradeapi.stream2.StreamConn(
    alpaca_api_key,
    alpaca_secret_key,
    base_url=alpaca_paper_endpoint,
    data_url=websocket_url,
    data_stream='alpacadatav1'
)

# Stream account updates:
@conn.on(r'^account_updates$')
async def on_account_updates(conn, channel, account):
    print('account', account)
    
# Stream trade updates:
@conn.on(r'^trade_updates$')
async def on_trade_updates(conn, channel, trade):
    print('trade', trade)

@conn.on(r'^T.AAPL$')
async def price_data(conn, channel, bar):
    print(bar.price)
    
def ws_start():
	conn.run(['account_updates', 'trade_updates'])

# Start the WebSocket thread:
ws_thread = threading.Thread(target=ws_start, daemon=True)
ws_thread.start()

In [ ]:
# Initiate empty variables:
stock_price = float('nan')
stock_sold_for = float()
is_stock_owned = 'false'
symbol = 'IPDN'

In [ ]:
# Check if market is open:
clock = alpaca_api.get_clock()
if clock.is_open:
	pass
else:
	time_to_open = clock.next_open - clock.timestamp
	sleep(time_to_open.total_seconds())

In [ ]:
# Check time til market close:
def time_to_market_close():
    clock = alpaca_api.get_clock()
    closing = clock.next_close - clock.timestamp
    return round(closing.total_seconds() / 60)

In [ ]:
time_to_market_close()

In [ ]:
# High frequency trading loop:
def hft(symbol, is_stock_owned, stock_price):
    for ticker in positions_tickers:
        current_price = alpaca_api.list_positions(symbol.current_price) 
        # check if we own the stock
        if is_stock_owned == 'false' and stock_price == 'nan':
            # buy stock
            alpaca_api.submit_order(
                symbol=symbol, qty=100, side='buy',
                type='limit', time_in_force='day',
                limit_price=str(alpaca_api.list_positions(symbol.current_price))
            )
            stock_price = alpaca_api.list_positions(symbol.current_price)
            is_stock_owned = 'true'
        if is_stock_owned == 'false' and stock_price != 'nan':
            if alpaca_api.list_positions(symbol.current_price) > stock_sold_for:
                # Buy stock
                alpaca_api.submit_order(
                symbol=symbol, qty=100, side='buy',
                type='limit', time_in_force='day',
                limit_price=str(alpaca_api.list_positions(symbol.current_price))
            )
        if is_stock_owned == 'false' and stock_sold_for < stock_price:
            next
            if alpaca_api.list_positions(symbol.current_price) > stock_sold_for:
                # Buy stock
                alpaca_api.submit_order(
                symbol=symbol, qty=100, side='buy',
                type='limit', time_in_force='day',
                limit_price=str(alpaca_api.list_positions(symbol.current_price))
            )
        if is_stock_owned == 'true' and (alpaca_api.list_positions(symbol.current_price) > stock_price):
            # do not sell stock if price is greater than or equal to purchase price
#             if alpaca_api.list_positions(symbol.current_price) >= stock_price:
                next
            # sell stock if current price is below what we paid for it
        if alpaca_api.list_positions(symbol.current_price) < stock_price:
            alpaca_api.submit_order(
                symbol=symbol, qty=100, side='sell',
                type='limit', time_in_force='day',
                limit_price = str(alpaca_api.list_positions(symbol.current_price))
            )
            stock_sold_for = float((alpaca_api.list_positions(symbol.current_price)))
            is_stock_owned = 'false'

In [ ]:
hft(symbol, is_stock_owned, stock_price)